In [46]:
from collections import namedtuple
from contextlib import contextmanager
import csv

# Section - 1: Context Manager + Generator using a Python Class

In [27]:
class CSVDataIterator:
    ''' Reads the CSV file, generates a "Generator" where data is parses in named tuple
    '''
    def __init__(self, file_name: str, mode: str, separator: str):
        ''' Initializes the class with properties required for reading the file
        
        Parameters:
        ----------
        file_name:  File Name in STRING, which to be read
        mode:       'r' for read, 'w' for write and 'a' for append. Although only 'r' is supported
        separator:  Seperator which seperates the data in each row. Generally it will be ',' 
                    but it can be any charecter
        
        '''
        if mode not in ['r']:
            raise ValueError(f"Mode {mode} is not supported. Only 'r' (read) mode is allowed.")

        self._file = file_name
        self._mode = mode
        self._separator = separator
        # Header of the CSV file is saved in this list. It is assumed that only one 
        # line of header is present in the CSV file
        self._header = []

    def __iter__(self):
        ''' Iterator for the "for" loop 
        '''
        return self
    
    def __next__(self):
        ''' To fetch the next time. Each row is fetched and then new line charecter is stripped
        and each row is then split according to "seperator" which was sending during construction 
        of the object 
        '''
        row = next(self._CSV_file_fd)
        data = self._csv_data(*row)
        return data

    def __enter__(self):
        ''' When enter the context, read the first line (which is header) and return the object
        '''
        try:
            self._fd = open(self._file, self._mode)
            self._CSV_file_fd = csv.reader(self._fd, delimiter= self._separator)
            self._header = list(next(self._CSV_file_fd))
            self._csv_data = namedtuple('csv_data', self._header)
            return self
        except IOError as e:
            print(f"Error opening file: {e}")
            raise
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        ''' Upon exiting the context close the file and display any error/exception occurred. 
        if exception has occured then stop the excecution of the code
        '''
        self._fd.close()
        if exc_type:
            print(f"Exception {exc_type} occurred: {exc_value}")
            
        if self._fd.closed:
            print("File closed properly")
        else:
            print("File did not close properly.")
        return False


### Try with 'cars-2.csv' and 'personal_info.csv' files and display first 5 items

In [31]:
num_of_rows_to_print = 0
with CSVDataIterator('cars-2.csv', 'r', ';') as file:
    print(file._header)
    for row in file:
        print (row)
        num_of_rows_to_print += 1
        if (num_of_rows_to_print > 5):
            break

['Car', 'MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model', 'Origin']
csv_data(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
csv_data(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
csv_data(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
csv_data(Car='AMC Rebel SST', MPG='16.0', Cylinders='8', Displacement='304.0', Horsepower='150.0', Weight='3433.', Acceleration='12.0', Model='70', Origin='US')
csv_data(Car='Ford Torino', MPG='17.0', Cylinders='8', Displacement='302.0', Horsepower='140.0', Weight='3449.', Acceleration='10.5', Model='70', Origin='US')
csv_data(Car='Ford Galaxie 500', MPG='15.0', Cylinders='8', Displacement=

In [33]:
num_of_rows_to_print = 0
with CSVDataIterator('personal_info.csv', 'r', ',') as file:
    print(file._header)
    for row in file:
        print (row)
        num_of_rows_to_print += 1
        if (num_of_rows_to_print > 5):
            break

['ssn', 'first_name', 'last_name', 'gender', 'language']
csv_data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
csv_data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
csv_data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
csv_data(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
csv_data(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')
csv_data(ssn='105-27-5541', first_name='Federico', last_name='Aggett', gender='Male', language='Chinese')
File closed properly


# Section - 2: Context Manager + Generator using a Python Function and ContextManager Decorator

In [59]:
@contextmanager
def CSVDataIterator_f(fname :str, mode :str = 'r', separator :str= ','):
    ''' Function helps in generating the "Generator" by reading a csv file.
    '''
    print("Trying to open file ... ")
    try:
        print(f"{fname} and {mode}")
        file_fd = open(fname, mode)
        csv_file_fd = csv.reader(file_fd, delimiter=separator)

        # Read the header explicitly
        header = next(csv_file_fd)
        print("Header:", header)

        # yield untill end of the file 
        yield csv_file_fd
    except FileNotFoundError:
        print("No such file exists in this path")
    finally:
        file_fd.close()


### Try with 'cars-2.csv' and 'personal_info.csv' files and display first 5 items

In [60]:
num_of_rows_to_print = 0
with CSVDataIterator_f('cars-2.csv', 'r', ';') as file:
    for row in file:
        print (row)
        num_of_rows_to_print += 1
        if (num_of_rows_to_print > 5):
            break

Trying to open file ... 
cars-2.csv and r
Header: ['Car', 'MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model', 'Origin']
['Chevrolet Chevelle Malibu', '18.0', '8', '307.0', '130.0', '3504.', '12.0', '70', 'US']
['Buick Skylark 320', '15.0', '8', '350.0', '165.0', '3693.', '11.5', '70', 'US']
['Plymouth Satellite', '18.0', '8', '318.0', '150.0', '3436.', '11.0', '70', 'US']
['AMC Rebel SST', '16.0', '8', '304.0', '150.0', '3433.', '12.0', '70', 'US']
['Ford Torino', '17.0', '8', '302.0', '140.0', '3449.', '10.5', '70', 'US']
['Ford Galaxie 500', '15.0', '8', '429.0', '198.0', '4341.', '10.0', '70', 'US']


In [61]:
num_of_rows_to_print = 0
with CSVDataIterator('personal_info.csv', 'r', ',') as file:
    print(file._header)
    for row in file:
        print (row)
        num_of_rows_to_print += 1
        if (num_of_rows_to_print > 5):
            break

['ssn', 'first_name', 'last_name', 'gender', 'language']
csv_data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
csv_data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
csv_data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
csv_data(ssn='104-22-0928', first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi')
csv_data(ssn='104-84-7144', first_name='Claudianus', last_name='Brixey', gender='Male', language='Afrikaans')
csv_data(ssn='105-27-5541', first_name='Federico', last_name='Aggett', gender='Male', language='Chinese')
File closed properly
